## 🧹 Tokenflow Dataset Preparation for Link Prediction

This notebook provides the necessary preprocessing steps to prepare the **Tokenflow** dataset for use with the **RelationalAI Graph Neural Network (GNN) learning engine**.

We specifically format the dataset for a **link prediction task**, where the goal is to predict relationships (edges) between entities in a graph.

Once the preprocessing is complete, the processed data will be uploaded to the appropriate **Snowflake tables**, ready to be consumed by the GNN engine.


In [15]:
import pandas as pd

## 🔗 Link Prediction Use Case: Predicting Transactions

In our link prediction task, the goal is to predict **transaction links** between **senders** and **buyers**.

To prepare the data for this task, we will follow these steps:

1. **Create Entity Tables**
   We will create two tables—one for **buyers** and one for **senders**—containing the unique IDs of each entity type.

2. **Create the Relationship (Edge) Table**
   We will use `token-trades.csv` as our **transaction table**, representing the edges (links) between senders and buyers.

3. **Generate Train, Validation, and Test Splits**
   Using the transaction data, we will create separate datasets for training, validation, and testing.

This setup ensures the data is properly formatted and ready for ingestion by the RelationalAI GNN engine.


In [16]:


trades_df = pd.read_csv('../../data/token-trades.csv')
trades_df.head()

,LAUNCHPAD,BLOCK_TIMESTAMP,TX_HASH,TX_SENDER_ADDRESS,POOL,SENDER,TAKER,BUY_TOKEN_ADDRESS,BUY_TOKEN_SYMBOL,BUY_TOKEN_NAME,BUY_AMOUNT,SELL_TOKEN_ADDRESS,SELL_TOKEN_SYMBOL,SELL_TOKEN_NAME,SELL_AMOUNT
0,VIRTUALS,2025-02-11 08:08:29.000,0x480bd3ace08ddf1bcc961610a445ee3285db8ede56a7...,0x5773381b7d56a21b4b70660afe1bbaa3c9443032,0xde40586745a4e21efc0e8e0073cdd431ae88876a,0xc7d3ab410d49b664d03fe5b1038852ac852b1b29,0xc7d3ab410d49b664d03fe5b1038852ac852b1b29,0x2676e4e0e2eb58d9bdb5078358ff8a3a964cedf5,POLY,Polytrader,118185645299910489396104,0x0b3e328455c4059eeb9e3f84b5543f74e24e7e1b,VIRTUAL,Virtual Protocol,351851592692643374539
1,CREATOR_BID,2025-01-05 02:39:57.000,0x56451552195b41641fe7374ce3a177734b819ef3d10d...,0x6e270ef8e600368f82fb286e4eee7aed7de4e3e2,0x3461ff03aca2abb3c4288e8fa4e7368bd42b7527,0x4752ba5dbc23f44d87826276bf6fd6b1c372ad24,0x6e270ef8e600368f82fb286e4eee7aed7de4e3e2,0x8a455c262602afe5f099a9b8f990025c87d365fe,TAKION,TAKION 7400,124882031399680511,0x4200000000000000000000000000000000000006,WETH,Wrapped Ether,1000000000000
2,VIRTUALS,2024-12-03 10:02:45.000,0x1e352c96433017c66300352d2bf4584a9d309049bda2...,0x5d1faef6cc6c2afccf652193a08470eda3156eb7,0x04dd95a4d0f9b6fc7f20ac0dcfcd8835f5bac5af,0x013bb8a204499523ddf717e0abaa14e6dc849060,0x5d1faef6cc6c2afccf652193a08470eda3156eb7,0xcc72f306de93e08c9ed4e7aaa00c2e4f818bcc8d,$QUAN,Quandale Dingle,1010761224665291045476216,0x0b3e328455c4059eeb9e3f84b5543f74e24e7e1b,VIRTUAL,Virtual Protocol,247385745714260043306
3,VIRTUALS,2024-12-04 15:19:15.000,0xcb056ddc022d47d7c7c0384af5d34db85c0e2247210d...,0xa8632dc3fe5525a9b0d7f65cf53a429b7175d172,0x93d65a935e7c0f1aa153780e0db3ad08b9448c89,0x013bb8a204499523ddf717e0abaa14e6dc849060,0xa8632dc3fe5525a9b0d7f65cf53a429b7175d172,0x7588880d9c78e81fade7b7e8dc0781e95995a792,SAINT,Satoshi AI agent,179380336030427456310882,0x0b3e328455c4059eeb9e3f84b5543f74e24e7e1b,VIRTUAL,Virtual Protocol,2240090278799805557408
4,VIRTUALS,2024-12-05 10:08:17.000,0x44f7250a1aa2e1d67b988ebc112826ccecd2bfcd266b...,0xa8632dc3fe5525a9b0d7f65cf53a429b7175d172,0xe23a29d610808acb58cd01860f90a7ec59a99832,0x4752ba5dbc23f44d87826276bf6fd6b1c372ad24,0x86cbac9d9ac726f729eef6627dc4817bcbb03a9c,0x0b3e328455c4059eeb9e3f84b5543f74e24e7e1b,VIRTUAL,Virtual Protocol,9302414725783243127,0x18c31cbff3e717c3bec29aaff613b9987e7d73a8,LARP,Agent Larp,153688397202799342467323


In [8]:
# create two dataframes with unique buyer and sender IDs
buyers_df = trades_df[['BUY_TOKEN_ADDRESS']].drop_duplicates()
senders_df = trades_df[['TX_SENDER_ADDRESS']].drop_duplicates()
# let's make a transaction table from the trades table by keeping some
# of the features
transactions_df = trades_df[['TX_SENDER_ADDRESS','BUY_TOKEN_ADDRESS','BLOCK_TIMESTAMP',
                             'BUY_AMOUNT','BUY_TOKEN_SYMBOL','SELL_TOKEN_SYMBOL',
                             'SELL_AMOUNT']]

In [17]:
transactions_df['BLOCK_TIMESTAMP'].max()

Timestamp('2025-02-18 06:37:49')

In [25]:
# create train, test and validation data
train_start_date = "2024-10-21"
train_end_date = "2025-01-14"
val_end_date = "2025-01-31"

# Ensure BLOCK_TIMESTAMP is in datetime format
transactions_df['BLOCK_TIMESTAMP'] = pd.to_datetime(transactions_df['BLOCK_TIMESTAMP'])

train_df = transactions_df[
    (transactions_df['BLOCK_TIMESTAMP'] >= train_start_date) &
    (transactions_df['BLOCK_TIMESTAMP'] <= train_end_date)
][['BLOCK_TIMESTAMP', 'TX_SENDER_ADDRESS', 'BUY_TOKEN_ADDRESS']]

val_df = transactions_df[
    (transactions_df['BLOCK_TIMESTAMP'] > train_end_date) &
    (transactions_df['BLOCK_TIMESTAMP'] <= val_end_date)
][['BLOCK_TIMESTAMP', 'TX_SENDER_ADDRESS', 'BUY_TOKEN_ADDRESS']]

test_df = transactions_df[
    (transactions_df['BLOCK_TIMESTAMP'] > val_end_date)
][['BLOCK_TIMESTAMP', 'TX_SENDER_ADDRESS', 'BUY_TOKEN_ADDRESS']]

# note that for link prediction tasks we expect that the destination
# entities - the entities that we are trying to predict a link to - 
# to be grouped in a list. So we will transform our trian,test and
# validation dataframes accordingly
train_df = train_df.groupby(['TX_SENDER_ADDRESS', 'BLOCK_TIMESTAMP'])['BUY_TOKEN_ADDRESS'].agg(list).reset_index()
val_df = val_df.groupby(['TX_SENDER_ADDRESS', 'BLOCK_TIMESTAMP'])['BUY_TOKEN_ADDRESS'].agg(list).reset_index()
test_df = test_df.groupby(['TX_SENDER_ADDRESS', 'BLOCK_TIMESTAMP'])['BUY_TOKEN_ADDRESS'].agg(list).reset_index()


print(f'Train size: {train_df.shape[0]}')
print(f'Validation size: {val_df.shape[0]}')
print(f'Test size: {test_df.shape[0]}')

# in link prediction problems, validation and test data need
# to all have the same timestamp, so we will fake it for now
val_df['BLOCK_TIMESTAMP'] =  pd.Timestamp("2025-01-31 13:23:59.000")
test_df['BLOCK_TIMESTAMP'] = pd.Timestamp("2025-02-01 13:23:59.000")

Train size: 16088
Validation size: 1435
Test size: 493


## Upload Data To Snowflake

We assume you have created a `.env` file in the same directory as this notebook. (A sample `.env` file is included for reference.)

Your `.env` file should contain the following fields:

```
ACCOUNT_NAME=snowflake_account_name
USER_NAME=snowflake_user_name
PASSWORD=snowflake_user_password
WAREHOUSE=snowflake_warehouse
APP_NAME=RAI_EXPT_APP
AUTH_METHOD=password
```

In [26]:
from load_to_snowflake import create_session, load_to_snowflake


In [27]:
import os
from dotenv import load_dotenv
load_dotenv()

snowflake_config = {
    "account": os.getenv("ACCOUNT_NAME"),
    "user": os.getenv("USER_NAME"),
    "password": os.getenv("PASSWORD"),
    "warehouse": os.getenv("WAREHOUSE"),
}

session = create_session(snowflake_config)

In [29]:
load_to_snowflake(
    session = session,
    df = buyers_df,
    database="GNN_TOKENFLOW",
    schema="DATA",
    table_name="BUYERS"
)

load_to_snowflake(
    session = session,
    df = senders_df,
    database="GNN_TOKENFLOW",
    schema="DATA",
    table_name="SENDERS"
)

load_to_snowflake(
    session = session,
    df = transactions_df,
    database="GNN_TOKENFLOW",
    schema="DATA",
    table_name="TRANSACTIONS"
)

load_to_snowflake(
    session = session,
    df = train_df,
    database="GNN_TOKENFLOW",
    schema="TASK",
    table_name="TRAIN"
)

load_to_snowflake(
    session = session,
    df = test_df,
    database="GNN_TOKENFLOW",
    schema="TASK",
    table_name="TEST"
)

load_to_snowflake(
    session = session,
    df = val_df,
    database="GNN_TOKENFLOW",
    schema="TASK",
    table_name="VALIDATION"
)

/Users/ilias/opt/anaconda3/envs/rai_gnn_env/lib/python3.11/site-packages/snowflake/snowpark/session.py:2721: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, _, _, ci_output = write_pandas(


Wrote data to GNN_TOKENFLOW.DATA.BUYERS
Wrote data to GNN_TOKENFLOW.DATA.SENDERS
Wrote data to GNN_TOKENFLOW.DATA.TRANSACTIONS
Wrote data to GNN_TOKENFLOW.TASK.TRAIN
Wrote data to GNN_TOKENFLOW.TASK.TEST
Wrote data to GNN_TOKENFLOW.TASK.VALIDATION
